In [43]:
# Due to the file system setting of the Jupyter Notebook, navigate to the package to use infra
%cd /home/jupyter/quant

/home/jupyter/quant


#### Create yfinance SQL tables for the first time

In [44]:
%run -i 'notebooks/etl/yfinance/create_sql_tables.py'

/opt/conda/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1050, "Table 'nasdaq_daily' already exists")
  result = self._query(query)


In [45]:
from datetime import datetime

### Extract and insert stock price datato cloud storage and SQL

#### Batch Load Max history data

In [46]:
ticker = 'AAPL'
password = 'jinyongwuxia'
table_name = 'nasdaq_daily'
period = 'max'

In [49]:
# Recent executed on
print(datetime.now())
print("======================================================================================================================================================")

%run -i 'notebooks/etl/yfinance/load_stock_price_daily.py'
print("======================================================================================================================================================")
    

2020-03-30 05:25:14.117392
[*********************100%***********************]  1 of 1 completed
AAPL in year 1980 has been write to gs://zhongyuan-dw/equity/company/aapl/1980/daily.csv
AAPL in year 1981 has been write to gs://zhongyuan-dw/equity/company/aapl/1981/daily.csv
AAPL in year 1982 has been write to gs://zhongyuan-dw/equity/company/aapl/1982/daily.csv
AAPL in year 1983 has been write to gs://zhongyuan-dw/equity/company/aapl/1983/daily.csv
AAPL in year 1984 has been write to gs://zhongyuan-dw/equity/company/aapl/1984/daily.csv
AAPL in year 1985 has been write to gs://zhongyuan-dw/equity/company/aapl/1985/daily.csv
AAPL in year 1986 has been write to gs://zhongyuan-dw/equity/company/aapl/1986/daily.csv
AAPL in year 1987 has been write to gs://zhongyuan-dw/equity/company/aapl/1987/daily.csv
AAPL in year 1988 has been write to gs://zhongyuan-dw/equity/company/aapl/1988/daily.csv
AAPL in year 1989 has been write to gs://zhongyuan-dw/equity/company/aapl/1989/daily.csv
AAPL in year 1

In [51]:
ts_pd.reset_index(inplace=True)

In [58]:
# Insert data into SQL table
# Set up cloud sql connections
connection = pymysql.connect(host=PRIVATE_HOST,
                             user=USER,
                             password=password,
                             db=DATABASE)

with connection.cursor() as cursor:
    for index, row in ts_pd.head(5).iterrows():
        sql = """
        INSERT INTO {table_name}({ticker}, {datetime}, {open}, {high}, {low}, {close}, {volume}, {year}, {month}, {day}, {updated_at}) 
        VALUES('{ticker_v}', '{datetime_v}', {open_v}, {high_v}, {low_v}, {close_v}, {volume_v}, {year_v}, {month_v}, {day_v}, '{updated_at_v}');
        """.format(table_name=table_name,
                   ticker=TICKER,
                   datetime=DATETIME,
                   open=OPEN, high=HIGH, low=LOW, close=CLOSE, volume=VOLUME,
                   year=YEAR, month=MONTH, day=DAY, updated_at=UPDATED_AT,
                   ticker_v=row[TICKER],
                   datetime_v=row[DATETIME],
                   open_v=row[OPEN], high_v=row[HIGH], low_v=row[LOW], close_v=row[CLOSE], volume_v=row[VOLUME],
                   year_v=row[YEAR], month_v=row[MONTH], day_v=row[DAY], updated_at_v=row[UPDATED_AT])

        print(sql)
        cursor.execute(sql)

        # connection is not autocommit by default. So you must commit to save your changes.
        connection.commit()

cursor.close()

datetime      1980-12-12 00:00:00
open                     0.406782
high                      0.40855
low                      0.406782
close                    0.406782
volume                  117258400
year                         1980
month                          12
day                            12
ticker                       AAPL
updated_at    2020-03-30 05:25:14
Name: 0, dtype: object

        INSERT INTO nasdaq_daily(ticker, datetime, open, high, low, close, volume, year, month, day, updated_at) 
        VALUES('AAPL', '1980-12-12 00:00:00', 0.40678155422210693, 0.4085501628991805, 0.40678155422210693, 0.40678155422210693, 117258400, 1980, 12, 12, '2020-03-30 05:25:14');
        
datetime      1980-12-15 00:00:00
open                     0.387327
high                     0.387327
low                      0.385558
close                    0.385558
volume                   43971200
year                         1980
month                          12
day                          